In [1]:
import os
from os import listdir
from os.path import isfile, join, splitext
import sys
import csv
import re

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# list shp files recursively
os.chdir('..')
abs_path = os.getcwd()

HIFLD_path = os.path.join(abs_path, 'data')
#csv_files = [os.path.join(root, name) \
#             for root, dirs, files in os.walk(HIFLD_path) \
#             for name in files \
#             if name.endswith(('.csv'))]
csv_files = [f for f in listdir(HIFLD_path) if isfile(join(HIFLD_path, f)) and f.endswith(".csv")]
csv_paths = [join(HIFLD_path, a) for a in csv_files]

In [3]:
# number of observations per geoshape file 
for file in csv_paths:
    with open(file, 'rb') as f:
        row_count = len(f.readlines()) - 1
        
        basename = os.path.basename(file).split('/')[0]
        fname = os.path.basename(basename).split('.')[0]
        
        print('There are {} observations in {}.'.format(row_count, fname))

There are 20000 observations in test_address_20k_v1.


# Parse Full Address
* Parse full address string to OMOP components
  * address_1
  * address_2
  * city
  * state
  * zip
  * county
  * location_source_value
  * latitude
  * longitude

In [4]:
# read all csv paths
test_list = []
for path in csv_paths:
    df = pd.read_csv(path)
    test_list.append(df)

test_addresses = pd.concat(test_list)

In [5]:
def parse_address(df, 
                  state_full_pattern,
                  state_abbr_pattern,
                  zip_code_pattern=r"[0-9]{5}(?:-[0-9]{4})?"
                  # state_code_pattern=r"[A-Z][a-z]+(?: +[A-Z][a-z]+)*)"
                 ):
    """
    Parse full address string to OMOP components by Regex search

    Parameters
    ----------
    df (DataFrame): Pandas DataFrame of source centroids with dates and user IDs

    Returns
    -------
    parse_df (DataFrame): DataFrame with parsed OMOP address components
    """
    tmp = []
    for i,row in df.iterrows():
        addr_components = row.Full_Addre.split(',')
    
        if len(re.findall(r'APT', row.Full_Addre, flags=re.IGNORECASE)) == 0:
            state_zip = addr_components[2].split(' ')
            if len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0:
                row['address_1'] = addr_components[0]
                row['address_2'] = np.NaN
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)   
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                row['address_1'] = addr_components[0]
                row['address_2'] = np.NaN
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = np.NaN
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0 and len(re.findall(state_full_pattern, addr_components[-1].title())) > 0:
                row['address_1'] = addr_components[0]
                row['address_2'] = np.NaN
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                row['address_1'] = addr_components[0]
                row['address_2'] = np.NaN
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())
                row['zip'] = np.NaN
    
                tmp.append(row)
        # elif 'APT' in str(row.Full_Addre) or 'Apt' in str(row.Full_Addre) or 'apt' in str(row.Full_Addre):
        elif len(re.findall(r'APT', row.Full_Addre, flags=re.IGNORECASE)) > 0:
            if len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0:
                base_address = addr_components[0]
                apt_string = re.findall(r'APT', row.Full_Addre, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(apt_string)[0]
                row['address_2'] = base_address.partition(apt_string)[1] + base_address.partition(apt_string)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                base_address = addr_components[0]
                apt_string = re.findall(r'APT', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(apt_string)[0]
                row['address_2'] = base_address.partition(apt_string)[1] + base_address.partition(apt_string)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = np.NaN
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0 and len(re.findall(state_full_pattern, addr_components[-1].title())) > 0:
                base_address = addr_components[0]
                apt_string = re.findall(r'APT', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(apt_string)[0]
                row['address_2'] = base_address.partition(apt_string)[1] + base_address.partition(apt_string)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                base_address = addr_components[0]
                apt_string = re.findall(r'APT', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(apt_string)[0]
                row['address_2'] = base_address.partition(apt_string)[1] + base_address.partition(apt_string)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())
                row['zip'] = np.NaN
    
                tmp.append(row)
                    
        # elif 'SUITE' in str(row.Full_Addre) or 'Suite' in str(row.Full_Addre):
        elif len(re.findall(r'SUITE', row.Full_Addre, flags=re.IGNORECASE)) > 0:
            if len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0:
                base_address = addr_components[0]
                suite_str = re.findall(r'SUITE', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(suite_str)[0]
                row['address_2'] = base_address.partition(suite_str)[1] + base_address.partition(suite_str)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]

                tmp.append(row)

            elif len(re.findall(state_abbr_pattern, addr_components[-1])) > 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                base_address = addr_components[0]
                suite_str = re.findall(r'SUITE', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(suite_str)[0]
                row['address_2'] = base_address.partition(suite_str)[1] + base_address.partition(suite_str)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_abbr_pattern, addr_components[-1])[0]
                row['zip'] = np.NaN
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) > 0 and len(re.findall(state_full_pattern, addr_components[-1].title())) > 0:
                base_address = addr_components[0]
                suite_str = re.findall(r'SUITE', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(suite_str)[0]
                row['address_2'] = base_address.partition(suite_str)[1] + base_address.partition(suite_str)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())[0]
                row['zip'] = re.findall(zip_code_pattern, addr_components[-1])[0]
    
                tmp.append(row)
    
            elif len(re.findall(state_abbr_pattern, addr_components[-1])) == 0 and len(re.findall(zip_code_pattern, addr_components[-1])) == 0:
                base_address = addr_components[0]
                suite_str = re.findall(r'SUITE', base_address, flags=re.IGNORECASE)[0]
                row['address_1'] = base_address.partition(suite_str)[0]
                row['address_2'] = base_address.partition(suite_str)[1] + base_address.partition(suite_str)[2]
                row['city'] = addr_components[1]
                row['state'] = re.findall(state_full_pattern, addr_components[-1].title())
                row['zip'] = np.NaN
    
                tmp.append(row)        
        
    return pd.DataFrame(tmp)

In [6]:
#states = ['IA', 'KS', 'UT', 'VA', 'NC', 'NE', 'SD', 'AL', 'ID', 'FM', 'DE', 'AK', 'CT', 'PR', 'NM', 'MS', 'PW', 'CO', 'NJ', 'FL', 'MN', 'VI', 'NV', 'AZ', 'WI', 'ND', 'PA', 'OK', 'KY', 'RI', 'NH', 'MO', 'ME', 'VT', 'GA', 'GU', 'AS', 'NY', 'CA', 'HI', 'IL', 'TN', 'MA', 'OH', 'MD', 'MI', 'WY', 'WA', 'OR', 'MH', 'SC', 'IN', 'LA', 'MP', 'DC', 'MT', 'AR', 'WV', 'TX']
#regex = re.compile(r'\b(' + '|'.join(states) + r')\b', re.IGNORECASE)
state_full_pattern = r"/AL|Alabama|AK|Alaska|AZ|Arizona|AR|Arkansas|CA|California|CO|Colorado|CT|Connecticut|DE|Delaware|FL|Florida|GA|Georgia|HI|Hawaii|ID|Idaho|IL|Illinois|IN|Indiana|IA|Iowa|KS|Kansas|KY|Kentucky|LA|Louisiana|ME|Maine|MD|Maryland|MA|Massachusetts|MI|Michigan|MN|Minnesota|MS|Mississippi|MO|Missouri|MT|Montana|NE|Nebraska|NV|Nevada|NH|New Hampshire|NJ|New Jersey|NM|New Mexico|NY|New York|NC|North Carolina|ND|North Dakota|OH|Ohio|OK|Oklahoma|OR|Oregon|PA|Pennsylvania|RI|Rhode Island|SC|South Carolina|SD|South Dakota|TN|Tennessee|TX|Texas|UT|Utah|VT|Vermont|VA|Virginia|WA|Washington|WV|West Virginia|WI|Wisconsin|WY|Wyoming/"
state_abbr_pattern = re.compile(r'\b(AZ|CA|...|NJ|N\.J\.|NM|N\.M\.|...)\b')
state_abbr_case = r"^([Aa][LKSZRAEPlkszraep]|[Cc][AOTaot]|[Dd][ECec]|[Ff][LMlm]|[Gg][AUau]|[Hh][Ii]|[Ii][ADLNadln]|[Kk][SYsy]|[Ll][Aa]|[Mm][ADEHINOPSTadehinopst]|[Nn][CDEHJMVYcdehjmvy]|[Oo][HKRhkr]|[Pp][ARWarw]|[Rr][Ii]|[Ss][CDcd]|[Tt][NXnx]|[Uu][Tt]|[Vv][AITait]|[Ww][AIVYaivy])$"
zip_code_pattern=r"[0-9]{5}(?:-[0-9]{4})?"

new_df = parse_address(test_addresses, state_full_pattern, state_abbr_pattern)

In [7]:
def clean_address(df):
    """
    Removes 'address_2' value if not exact match for 'APT' on RegEx match

    Parameters
    ----------
    df (DataFrame): Pandas DataFrame of parsed addresses from parse_address()

    Returns
    -------
    df (DataFrame): with cleaned 'address_2' field
    """
    df['address_2'] = np.where(df.address_2.str.match('^APT ', flags=re.IGNORECASE), df.address_2, np.NaN)

    return df

In [8]:
new_df = clean_address(new_df)
new_df.loc[new_df.address_2.notna()]

,Full_Addre,source_lon,source_lat,geometry,user_id,start_date,end_duration,end_date,address_1,address_2,city,state,zip
15,"4143 43RD ST APT E8, SUNNYSIDE, NY 11104",-73.920907,40.746160,POINT (-73.92090699984789 40.74615999998748),4,2008-01-12,417 days,2009-03-04,4143 43RD ST,APT E8,SUNNYSIDE,NY,11104
1152,"200 HANCOCK ST APT 904, BANGOR, ME 4401",-68.762050,44.802406,POINT (-68.76204999998546 44.80240600016944),290,2000-10-07,619 days,2002-06-18,200 HANCOCK ST,APT 904,BANGOR,ME,NaN
1265,"100 BOATSWAINS WAY APT 404, CHELSEA, MA 2150",-71.047452,42.390572,POINT (-71.04745200000409 42.390571999824786),316,2003-03-12,945 days,2005-10-12,100 BOATSWAINS WAY,APT 404,CHELSEA,MA,NaN
1420,"10 CONGRESS SQUARE PLZ APT 508, PORTLAND, ME 4101",-70.263132,43.654049,POINT (-70.26313200017458 43.654049000067545),355,2018-01-04,421 days,2019-03-01,10 CONGRESS SQUARE PLZ,APT 508,PORTLAND,ME,NaN
2434,"2353 MASSACHUSETTS AVE APT 66, CAMBRIDGE, MA 2140",-71.129031,42.396372,POINT (-71.12903099964991 42.39637200031826),623,2003-11-25,424 days,2005-01-22,2353 MASSACHUSETTS AVE,APT 66,CAMBRIDGE,MA,NaN
3034,"75 MORTON VILLAGE DR APT 408, MATTAPAN, MA 2126",-71.085149,42.280395,POINT (-71.08514899960493 42.28039500021873),768,2020-07-14,570 days,2022-02-04,75 MORTON VILLAGE DR,APT 408,MATTAPAN,MA,NaN
3347,"8 CHAPIN ST APT 2, WORCESTER, MA 1604",-71.790948,42.251026,POINT (-71.790947999763 42.25102599967064),852,2014-12-30,972 days,2017-08-28,8 CHAPIN ST,APT 2,WORCESTER,MA,NaN
3595,"1 EAST ST APT 2, ESSEX JCT, VT 5452",-73.107181,44.489873,POINT (-73.10718100043403 44.48987299991821),916,2019-07-02,554 days,2021-01-06,1 EAST ST,APT 2,ESSEX JCT,VT,NaN
3599,"523 RIDGE ST APT 3, FALL RIVER, MA 2724",-71.162535,41.690422,POINT (-71.16253500037273 41.69042200025634),917,2014-01-16,438 days,2015-03-30,523 RIDGE ST,APT 3,FALL RIVER,MA,NaN
3928,"5 W 86TH ST APT 14C, NEW YORK, NY 10024",-73.969960,40.785597,POINT (-73.96996000001063 40.785597000160934),1005,2002-08-22,599 days,2004-04-12,5 W 86TH ST,APT 14C,NEW YORK,NY,10024


# Parse with Lambda
* IGNORE THIS SECTION

In [88]:
test_addresses['state'] = test_addresses.Full_Addre.apply(lambda x: re.findall(state_abbr_pattern, x))

In [89]:
test_addresses

,Full_Addre,source_lon,source_lat,geometry,user_id,start_date,end_duration,end_date,state
0,"2499 SOUTH WILMINGTON AVENUE, COMPTON, CA 90220",-118.236303,33.868428,POINT (-118.23630253521881 33.86842825057474),1,2004-12-09,847 days,2007-04-05,[CA]
1,"420 South Pierre Street, Pierre, South Dakota ...",-98.695554,43.254257,POINT (-98.6955539999999 43.2542570000001),1,2006-02-04,527 days,2007-07-16,[420]
2,"12150 30 MILE ROAD, WASHINGTON, MI 48095",-83.013787,42.771167,POINT (-83.01378666886954 42.77116730707756),1,2018-07-12,949 days,2021-02-15,"[ 30, MI ]"
3,"65 TIFTON ELDORADO RD, TIFTON, GA 31794",-83.496325,31.445218,POINT (-83.49632463429873 31.445218156335972),2,2001-05-11,824 days,2003-08-13,"[65 , RD, GA ]"
4,"12512 WALTERS RD, HOUSTON, TX 77014",-95.468105,29.973519,POINT (-95.46810520662352 29.973518593722815),2,2004-01-09,924 days,2006-07-21,"[ RD, TX ]"
...,...,...,...,...,...,...,...,...,...
19995,"730 Reems Creek Road, Weaverville, NC 28787",-82.509238,35.694465,POINT (-82.50923810689038 35.6944646159304),4999,2006-04-17,628 days,2008-01-05,"[730, NC ]"
19996,"5793 South 700 East, Whitestown, IN 46075",-86.335915,39.955682,POINT (-86.33591469213522 39.95568248243973),4999,2010-05-21,463 days,2011-08-27,"[700, IN ]"
19997,"7150 E ARCH RD, STOCKTON, CA 95205",-121.187052,37.902070,POINT (-121.1870524708539 37.90207040341747),4999,2012-09-17,930 days,2015-04-05,"[ E , RD, CA]"
19998,"1075 GARRISONVILLE ROAD, STAFFORD, VA 22556",-77.469689,38.474291,POINT (-77.46968895783967 38.474291425596675),4999,2016-07-19,727 days,2018-07-16,[VA ]


In [79]:
state_abbr_pattern = re.compile(r'\b(AZ|CA|...|NJ|N\.J\.|NM|N\.M\.|...)\b')

re.findall(state_abbr_pattern, '2499 SOUTH WILMINGTON AVENUE, COMPTON, CA 90220')

['CA']

In [84]:
re.findall(state_full_pattern, '420 South Pierre Street, Pierre, South Dakota'.title())

['South Dakota']

In [48]:
state_full_pattern = r"/AL|Alabama|AK|Alaska|AZ|Arizona|AR|Arkansas|CA|California|CO|Colorado|CT|Connecticut|DE|Delaware|FL|Florida|GA|Georgia|HI|Hawaii|ID|Idaho|IL|Illinois|IN|Indiana|IA|Iowa|KS|Kansas|KY|Kentucky|LA|Louisiana|ME|Maine|MD|Maryland|MA|Massachusetts|MI|Michigan|MN|Minnesota|MS|Mississippi|MO|Missouri|MT|Montana|NE|Nebraska|NV|Nevada|NH|New Hampshire|NJ|New Jersey|NM|New Mexico|NY|New York|NC|North Carolina|ND|North Dakota|OH|Ohio|OK|Oklahoma|OR|Oregon|PA|Pennsylvania|RI|Rhode Island|SC|South Carolina|SD|South Dakota|TN|Tennessee|TX|Texas|UT|Utah|VT|Vermont|VA|Virginia|WA|Washington|WV|West Virginia|WI|Wisconsin|WY|Wyoming/"
state_abbr_pattern = re.compile(r'\b(AZ|CA|...|NJ|N\.J\.|NM|N\.M\.|...)\b')
state_abbr_case = r"^([Aa][LKSZRAEPlkszraep]|[Cc][AOTaot]|[Dd][ECec]|[Ff][LMlm]|[Gg][AUau]|[Hh][Ii]|[Ii][ADLNadln]|[Kk][SYsy]|[Ll][Aa]|[Mm][ADEHINOPSTadehinopst]|[Nn][CDEHJMVYcdehjmvy]|[Oo][HKRhkr]|[Pp][ARWarw]|[Rr][Ii]|[Ss][CDcd]|[Tt][NXnx]|[Uu][Tt]|[Vv][AITait]|[Ww][AIVYaivy])$"
zip_code_pattern=r"[0-9]{5}(?:-[0-9]{4})?"

test_addresses['zip'] = test_addresses.Full_Addre.apply(lambda x: re.findall(zip_code_pattern, x)[0] if len(re.findall(zip_code_pattern, x))>0 else np.NaN)
test_addresses['state'] = test_addresses.Full_Addre.apply(lambda x: re.findall(state_abbr_case, x)[0] if len(re.findall(state_abbr_case, x))>0 else (re.findall(state_full_pattern, x.title())[0] if len(re.findall(state_full_pattern, x.title()))>0 else np.NaN))
# test_addresses['state'] = test_addresses.Full_Addre.apply(lambda x: re.findall(state_full_pattern, x.title())[0] if len(re.findall(state_full_pattern, x.title()))>0 else np.NaN)
# test_addresses['state'] = test_addresses.Full_Addre.apply(lambda x: re.findall(state_abbr_pattern, x[0] if (len(re.findall(state_abbr_pattern, x)>0) and (np.isnan(x)) else np.NaN)

SyntaxError: invalid syntax (2629627975.py, line 9)

In [46]:
test_addresses

,Full_Addre,source_lon,source_lat,geometry,user_id,start_date,end_duration,end_date,zip,state
0,"2499 SOUTH WILMINGTON AVENUE, COMPTON, CA 90220",-118.236303,33.868428,POINT (-118.23630253521881 33.86842825057474),1,2004-12-09,847 days,2007-04-05,90220,NaN
1,"420 South Pierre Street, Pierre, South Dakota ...",-98.695554,43.254257,POINT (-98.6955539999999 43.2542570000001),1,2006-02-04,527 days,2007-07-16,57501,South Dakota
2,"12150 30 MILE ROAD, WASHINGTON, MI 48095",-83.013787,42.771167,POINT (-83.01378666886954 42.77116730707756),1,2018-07-12,949 days,2021-02-15,12150,Washington
3,"65 TIFTON ELDORADO RD, TIFTON, GA 31794",-83.496325,31.445218,POINT (-83.49632463429873 31.445218156335972),2,2001-05-11,824 days,2003-08-13,31794,NaN
4,"12512 WALTERS RD, HOUSTON, TX 77014",-95.468105,29.973519,POINT (-95.46810520662352 29.973518593722815),2,2004-01-09,924 days,2006-07-21,12512,NaN
...,...,...,...,...,...,...,...,...,...,...
19995,"730 Reems Creek Road, Weaverville, NC 28787",-82.509238,35.694465,POINT (-82.50923810689038 35.6944646159304),4999,2006-04-17,628 days,2008-01-05,28787,NaN
19996,"5793 South 700 East, Whitestown, IN 46075",-86.335915,39.955682,POINT (-86.33591469213522 39.95568248243973),4999,2010-05-21,463 days,2011-08-27,46075,NaN
19997,"7150 E ARCH RD, STOCKTON, CA 95205",-121.187052,37.902070,POINT (-121.1870524708539 37.90207040341747),4999,2012-09-17,930 days,2015-04-05,95205,NaN
19998,"1075 GARRISONVILLE ROAD, STAFFORD, VA 22556",-77.469689,38.474291,POINT (-77.46968895783967 38.474291425596675),4999,2016-07-19,727 days,2018-07-16,22556,NaN


In [38]:
def parse_address(df, zip_code_pattern, state_abbr_pattern, state_full_pattern):
    df['zip'] = df.Full_Addre.apply(lambda x: re.findall(zip_code_pattern, x)[0] if len(re.findall(zip_code_pattern, x))>0 else np.NaN)

    addr_components = row.Full_Addre.split(',')

In [40]:
re.findall(state_abbr_case, test_addresses.Full_Addre[0])

[]

In [11]:
apt_string = re.findall(r'APT', 'APT 610B', flags=re.IGNORECASE)
print(apt_string)

['APT']
